Testing derivaties for 1D MT problem.

Especially the rx.projectFieldsDeriv

In [1]:
import SimPEG as simpeg
import simpegEM as simpegem, simpegMT as simpegmt
from SimPEG.Utils import meshTensor
import numpy as np

In [2]:
# Setup the problem
sigmaHalf = 1e-2
# Frequency
nFreq = 33
# freqs = np.logspace(3,-3,nFreq)
freqs = np.array([100])
# Make the mesh
ct = 5
air = meshTensor([(ct,25,1.3)])
# coreT0 = meshTensor([(ct,15,1.2)])
# coreT1 = np.kron(meshTensor([(coreT0[-1],15,1.3)]),np.ones((7,)))
core = np.concatenate( (  np.kron(meshTensor([(ct,15,-1.2)]),np.ones((10,))) , meshTensor([(ct,20)]) ) )
bot = meshTensor([(core[0],10,-1.3)])
x0 = -np.array([np.sum(np.concatenate((core,bot)))])
# Change to use no air
m1d = simpeg.Mesh.TensorMesh([np.concatenate((bot,core))], x0=x0)
# Make the model
sigma = np.zeros(m1d.nC) + sigmaHalf
sigma[ m1d.gridCC > 0 ] = 1e-8

rxList = []
for rxType in ['z1dr','z1di']:
    rxList.append(simpegmt.SurveyMT.RxMT(simpeg.mkvc(np.array([0.0]),2).T,rxType))
# Source list
srcList =[]
tD = False
if tD:
    for freq in freqs:
        srcList.append(simpegmt.SurveyMT.srcMT_polxy_1DhomotD(rxList,freq))
else:
    for freq in freqs:
        srcList.append(simpegmt.SurveyMT.srcMT_polxy_1Dprimary(rxList,freq,sigma))
# Make the survey
survey = simpegmt.SurveyMT.SurveyMT(srcList)

# Set the problem
problem = simpegmt.ProblemMT1D.eForm_psField(m1d)
problem.pair(survey)

# Get the fields
fields = problem.fields(sigma)

# Project the data
data = survey.projectFields(fields)


Project at freq: 1.000e+02


We need calculate this derivative. 
\begin{align}
\underbrace{\frac{\partial P(f(u(m)),m^{fix})}{\partial f}}_{Rx}
\end{align}

Use the rule
\begin{align}
\frac{d}{dx}\left( \frac{a(x)}{b(x)} \right)  = \frac{\frac{d }{dx} a(x)  b(x) - a(x)\frac{d }{dx} b(x)  }{ b(x)^2 }
\end{align}

In the case of the 1D MT problem the data is calculated as 
\begin{align}
MT1Ddata = P(f(m)) &= \frac{P_{ex} f_e(src,m)}{P_{bx} f_b(src,m) \frac{1}{\mu_0}} = \frac{P_e u}{P_b f_b(u)} \\
\frac{\partial P(f(m))}{\partial u} v &=  \frac{P_e}{P_b \frac{1}{mu_0} f_b(u)}v - \frac{P_e u}{\left(P_b \frac{1}{mu_0} f_b(u)\right)^2} P_b \frac{1}{mu_0} \frac{d f_b}{du} v
\end{align}
where u is the fields that we solve for. 
\begin{align}
\frac{d f_b}{du} = - \frac{1}{i \omega} \nabla 
\end{align}



In [3]:
# Unused code &= \frac{ P_{ex} P_{bx} \frac{1}{\mu_0} \left( f_b(src,m) - f_e(src,m) \right) } { \left(P_{bx}f_b(src,m) \frac{1}{\mu_0} \right)^2 }

As matrices the formulas above can be written as
\begin{align}
\left[ \frac{\partial P(f(m))}{\partial u} v \right] = diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right] [P_e v] - diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right]^T diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right] \left[ P_b   \frac{d f_b}{du}(v) \frac{1}{mu_0} \right]
\end{align}



The adjoint problem is done simliarly
\begin{align}
\left[ \frac{\partial P(f(m))}{\partial u} v \right]^T = [P_e v]^T diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right]^T  - \left[ P_b   \frac{d f_b}{du}(v) \frac{1}{mu_0} \right]^T diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right] diag \left[ \frac{1}{\left(P_b \frac{1}{mu_0} f_b(u)\right)} \right]^T 
\end{align}


In [4]:
# def projectFields(self, src, mesh, u):
#         '''
#         Project the fields and return the
#         '''

#         if self.projType is 'Z1D':
#             Pex = mesh.getInterpolationMat(self.locs,'Fx')
#             Pbx = mesh.getInterpolationMat(self.locs,'Ex')
#             ex = Pex*mkvc(u[src,'e_1d'],2)
#             bx = Pbx*mkvc(u[src,'b_1d'],2)/mu_0
#             f_part_complex = ex/bx
#         real_or_imag = self.projComp
#         f_part = getattr(f_part_complex, real_or_imag)
#         return f_part

In [5]:
# Initate things for the derivs Test
src = survey.srcList[0]
rx = src.rxList[0]
u0 = np.random.randn(m1d.nN)+np.random.randn(m1d.nN)*1j
f0 = problem.fieldsPair(m1d,survey)
f0[src,'e_1d'] = u0
f0[src,'b_1d'] = -1/(1j*simpegem.Utils.EMUtils.omega(src.freq))*m1d.nodalGrad*u0

In [6]:
u0

array([ 0.36132584-1.64741907j, -1.09543652-0.5983257j ,
       -0.37399249-0.17183374j, -0.29875616+0.13386035j,
       -0.13497219-1.67592883j, -0.43805630+0.79351159j,
        1.10360642-1.88408115j, -1.55655836+0.09909074j,
        1.09262762-0.26644991j,  1.00456242+1.36030358j,
        0.10649574-0.2738965j , -0.93711934-0.82724463j,
        1.86172288-1.39322063j,  0.17645120-2.38628835j,
        0.68232151-0.96482639j, -1.54748216+0.02907389j,
       -0.33109688+0.19681238j, -0.70235831-1.00115598j,
       -1.13598355+0.21054259j, -0.09009229-0.07819883j,
       -0.01097127+0.68165894j,  0.52661676+0.27378467j,
       -0.70418270-1.10247881j, -0.51995781-0.20521301j,
       -0.12731995-0.33125501j, -1.59215935-2.21034752j,
       -0.18124185+0.32745866j,  0.67431311+1.33705062j,
        0.92167361-0.38476494j,  0.81511489-0.31198285j,
       -0.34442259+0.957995j  , -0.18007675-0.11694359j,
       -0.72309201+0.17539006j,  0.61922884+0.50879561j,
        0.47818118+0.81750598j,

In [7]:
# Run a test
def fun(u):
    f = problem.fieldsPair(m1d,survey)
    f[src,'e_1d'] = u
    f[src,'b_1d'] = -(m1d.nodalGrad*u)/(1j*simpegem.Utils.EMUtils.omega(src.freq))
    return rx.projectFields(src,m1d,f), lambda t: rx.projectFieldsDeriv(src,m1d,f0,t)
simpeg.Tests.checkDerivative(fun,u0,num=4,plotIt=False)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    1.969e-05     8.331e-07      nan
 1   1.00e-02    2.045e-06     7.979e-09      2.019
 2   1.00e-03    2.052e-07     7.945e-11      2.002
 3   1.00e-04    2.052e-08     7.942e-13      2.000
========================= PASS! =========================
That was easy!



True

In [8]:
print rx.projectFieldsDeriv(src,m1d,f0,u0)
print m1d.nF
print m1d.nN

[ -8.67361738e-19]
181
181


In [9]:
# fields._b_1dDeriv_u(src,u0)

In [10]:
survey.dpred(f0)

TypeError: object of type 'FieldsMT' has no len()